In [17]:
import os
import json
import pandas as pd
import datetime
import pprint

# Merge Datasets
We want to merge 2 datasets, gm_history.csv and trades.json, to get a list of trades between GMs

To do this, we will loop through all trades extracted in trades.json, and append the GMs for each team at the date the trade happened.

#### 1. Read in datasets

In [59]:
path = os.getcwd()
path = os.path.abspath(os.path.join(path, os.pardir, 'data/trades.json'))
with open(path) as f:
    trades = json.load(f)
    
path = os.getcwd()
path = os.path.abspath(os.path.join(path, os.pardir, 'data/gm_history.csv'))
with open(path) as f:
    df_gm_hist = pd.read_csv(f)

clean up trades dataset, replace 'Colorado Avalanch' with 'Colorado Avalanche'

In [60]:
count = 0
errors = 0
for year in trades:
    count += len(trades[year])
    for trade in trades[year]:
        if trade['team1'] == 'Colorado Avalanch':
            trade['team1'] = 'Colorado Avalanche'
            errors += 1
        if trade['team2'] == 'Colorado Avalanch':
            trade['team2'] = 'Colorado Avalanche'
            errors += 1
print('total errors fixed: ' + str(errors))
print('total trades: ' + str(count))

total errors fixed: 119
total trades: 2210


clean up df_gm_hist to recognize date types

In [62]:
df = df_gm_hist[['team', 'start_date', 'name']]
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df['start_date'] = pd.to_datetime(df['start_date'], format="%B %d, %Y", errors='coerce')

In [63]:
# pd.set_option("display.max_rows", None, "display.max_columns", None)
# df

#### 2. Merge dataset (create new dataset where for each trade, team data is replaced by team GMs)

In [69]:
for year in trades:
    for trade in trades[year]:
        date = datetime.datetime.strptime(trade['date'], '%d%b%Y')
        try:
            gm_1 = df[ (df['team']==trade['team1']) & (df['start_date'] < date) ].iloc[-1]['name']
            gm_2 = df[ (df['team']==trade['team2']) & (df['start_date'] < date) ].iloc[-1]['name']
            trade['team1_gm'] = gm_1
            trade['team2_gm'] = gm_2
        except Exception:
            pprint.pprint(trade)

    #     print(date)
    #     print(trade['team1'] + ': ' + gm_1)
    #     print(trade['team2'] + ': ' + gm_2)
    #     print('\n\n')
    
# save to json file in data directory
path = os.getcwd()
path = os.path.abspath(os.path.join(path, os.pardir, 'data/trades_and_gms.json'))
with open(path, 'w') as f:
        json.dump(trades, f, indent=2)

In [70]:
# for trade in trades['2014-15']:
#     print(trade['date'])
#     print(trade['team1_gm'])
#     print(trade['team2_gm'])
#     print()